In [ ]:
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.metrics import r2_score
# import matplotlib.pyplot as plt

# # ---------- 1. SKLEARN PART (you already had most of this) ----------

# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.model_selection import train_test_split

# # target + features
# y = df["annual_premium"]
# X = df.drop(columns=[
#     "annual_premium",
#     "annual_medical_cost",
#     "monthly_premium",
#     "avg_claim_amount",
#     "total_claims_paid"
# ])

# # split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# # detect columns FROM X
# categorical_cols = X_train.select_dtypes(include=["object"]).columns
# numeric_cols = X_train.select_dtypes(exclude=["object"]).columns

# # preprocessor
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
#         ("num", StandardScaler(), numeric_cols),
#     ],
#     remainder="drop",
# )

# # fit preprocessor on train
# preprocessor.fit(X_train)

# # transform to numeric
# X_train_enc = preprocessor.transform(X_train)
# X_test_enc = preprocessor.transform(X_test)

# # ColumnTransformer with OneHotEncoder often returns sparse matrices → make dense
# if hasattr(X_train_enc, "toarray"):
#     X_train_enc = X_train_enc.toarray()
#     X_test_enc = X_test_enc.toarray()

# # to numpy float32
# X_train_np = X_train_enc.astype(np.float32)
# X_test_np = X_test_enc.astype(np.float32)
# y_train_np = y_train.to_numpy(dtype=np.float32).reshape(-1, 1)
# y_test_np = y_test.to_numpy(dtype=np.float32).reshape(-1, 1)

# # ---------- 2. TORCH DATASETS / LOADERS ----------

# train_ds = TensorDataset(torch.from_numpy(X_train_np), torch.from_numpy(y_train_np))
# test_ds = TensorDataset(torch.from_numpy(X_test_np), torch.from_numpy(y_test_np))

# train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
# test_loader = DataLoader(test_ds, batch_size=256)

# # ---------- 3. MODEL ----------

# class MLP(nn.Module):
#     def __init__(self, in_features):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(in_features, 128),
#             nn.ReLU(),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.Linear(64, 1),  # regression
#         )

#     def forward(self, x):
#         return self.net(x)

# device = (
#     torch.device("mps")
#     if torch.backends.mps.is_available()
#     else torch.device("cuda" if torch.cuda.is_available() else "cpu")
# )

# model = MLP(in_features=X_train_np.shape[1]).to(device)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# # ---------- 4. TRAIN LOOP WITH LOSS TRACKING ----------

# train_losses = []
# val_losses = []

# for epoch in range(30):
#     model.train()
#     running_loss = 0.0
#     for xb, yb in train_loader:
#         xb = xb.to(device)
#         yb = yb.to(device)

#         preds = model(xb)
#         loss = criterion(preds, yb)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * xb.size(0)

#     train_loss = running_loss / len(train_loader.dataset)

#     # validation
#     model.eval()
#     val_loss = 0.0
#     all_preds = []
#     all_true = []
#     with torch.no_grad():
#         for xb, yb in test_loader:
#             xb = xb.to(device)
#             yb = yb.to(device)
#             preds = model(xb)
#             loss = criterion(preds, yb)

#             val_loss += loss.item() * xb.size(0)
#             all_preds.append(preds.cpu().numpy())
#             all_true.append(yb.cpu().numpy())

#     val_loss /= len(test_loader.dataset)

#     train_losses.append(train_loss)
#     val_losses.append(val_loss)

#     print(f"Epoch {epoch+1:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")


# # ---------- 6. PLOT LOSSES ----------

# plt.figure(figsize=(7, 4))
# plt.plot(train_losses, label="Train Loss")
# plt.plot(val_losses, label="Validation Loss")
# plt.xlabel("Epoch")
# plt.ylabel("MSE Loss")
# plt.title("Train vs Validation Loss")
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
# ---------- Evaluate final model ----------
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        preds = model(xb)
        all_preds.append(preds.cpu().numpy())
        all_true.append(yb.cpu().numpy())

y_true = np.concatenate(all_true).ravel()
y_pred = np.concatenate(all_preds).ravel()

r2 = r2_score(y_true, y_pred)
print(f"\nFinal R² score on test set: {r2:.4f}")

We define our feature matrix and target variable

In [ ]:
# #choosing features
# Feature_matrix = ['age', 'sex', 'region', 'urban_rural', 'education', 'employment_status',
#        'bmi', 'smoker', 'alcohol_freq', 'visits_last_year',
#        'hospitalizations_last_3yrs', 'days_hospitalized_last_3yrs',
#        'medication_count', 'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c',
#        'plan_type', 'risk_score', 'chronic_count', 'hypertension', 'diabetes', 'asthma',
#        'copd', 'cardiovascular_disease', 'cancer_history', 'kidney_disease',
#        'liver_disease', 'arthritis', 'mental_health', 'proc_imaging_count',
#        'proc_surgery_count', 'proc_physio_count', 'proc_consult_count', 'is_high_risk', 'had_major_procedure']

# #defining X og y
# X = df[Feature_matrix]
# y = df["annual_premium"]

# #changing categories to numerical
# X = pd.get_dummies(X, drop_first=True)

# print("X shape:", X.shape)
# print("y shape:", y.shape)
# display(X.head())
# display(y.head())

LINEAR REGRESSION

In [ ]:
# # fitting a linear regression model to X and y 
# lr = LinearRegression()
# lr.fit(X,y)

In [ ]:
# #printing the score
# print("Score: {:.3f}".format(lr.score(X, y)))

In [ ]:
#printing the coefficient
coefficient = lr.coef_
print(coefficient)